Predicting Membrane Permeability in PAMPA Assay

In [ ]:
!pip install PyTDC
!pip install rdkit

In [ ]:
import pandas as pd
import numpy as np
from tdc.single_pred import ADME
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import EState
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import GraphDescriptors
from rdkit.Chem import Crippen
from rdkit.Chem import QED

In [ ]:
data = ADME(name = 'PAMPA_NCATS')
split = data.get_split()

Downloading...
100%|██████████| 144k/144k [00:00<00:00, 1.64MiB/s]
Loading...
Done!


In [ ]:
train_df = split['train']
valid_df = split['valid']
test_df = split['test']

In [ ]:
def calculate_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    descriptors = {}
    for descriptor_name, function in Descriptors.descList:
        descriptors[descriptor_name] = function(mol)
    return descriptors

In [ ]:
# Calculate descriptors for train, valid, and test sets
train_df['Descriptors'] = train_df['Drug'].apply(calculate_descriptors)
valid_df['Descriptors'] = valid_df['Drug'].apply(calculate_descriptors)
test_df['Descriptors'] = test_df['Drug'].apply(calculate_descriptors)

In [ ]:
# Prepare features and labels
X_train = pd.DataFrame(train_df['Descriptors'].tolist())
y_train = train_df['Y']

X_valid = pd.DataFrame(valid_df['Descriptors'].tolist())
y_valid = valid_df['Y']

X_test = pd.DataFrame(test_df['Descriptors'].tolist())
y_test = test_df['Y']

In [ ]:
print(split['train'].head())

   Drug_ID                                               Drug  Y  \
0  1259573            COC1=C(C=C(C=C1)CCN2C(=CC(=O)NC2=S)N)OC  0   
1  1275864  COC1=C(C=C(C=C1)Cl)C(=O)NC2=CC=C(C=C2)NC(=O)C3...  1   
2  2030130  CN1C2=CC=CC=C2C(=O)C3=C1N=C(N(C3=O)C4=CC=CC=C4...  1   
3  4422695  CC1=C(C=C(C=C1)NS(=O)(=O)C2=CC=CC(=C2)C(=O)O)S...  1   
4  1131802  COC1=CC(=CC(=C1O)OC)C2=NC(=C(N2)C3=CC=CS3)C4=C...  1   

                                         Descriptors  
0  {'MaxAbsEStateIndex': 11.245651352831256, 'Max...  
1  {'MaxAbsEStateIndex': 12.432253765537947, 'Max...  
2  {'MaxAbsEStateIndex': 13.718748818972035, 'Max...  
3  {'MaxAbsEStateIndex': 12.943376269393585, 'Max...  
4  {'MaxAbsEStateIndex': 10.16924146206794, 'MaxE...  


In [ ]:
print(len(train_df['Descriptors'][0]))

210


In [ ]:
from sklearn.impute import SimpleImputer

# Impute missing values with mean
imputer = SimpleImputer(strategy='mean')
X_valid_imputed = imputer.fit_transform(X_valid)

# Convert X_valid_imputed to a DataFrame
X_valid_imputed_df = pd.DataFrame(X_valid_imputed, columns=X_valid.columns)

In [ ]:
# Concatenate training and validation datasets
X_combined = pd.concat([X_train, X_valid_imputed_df], axis=0)
y_combined = pd.concat([y_train, y_valid], axis=0)

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, confusion_matrix, cohen_kappa_score, accuracy_score

rf_classifier = RandomForestClassifier(n_estimators=200)

# Perform cross-validation
cv_scores = cross_val_score(rf_classifier, X_combined, y_combined, cv=5, scoring='accuracy')

# Print cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())

In [ ]:
# Train the model
rf_classifier.fit(X_combined, y_combined)

In [ ]:
def evaluate_model(model, X, y):
    y_pred = model.predict(X)
    auc_roc = roc_auc_score(y, y_pred)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    balanced_accuracy = (sensitivity + specificity) / 2
    kappa = cohen_kappa_score(y, y_pred)
    accuracy = accuracy_score(y, y_pred)

    return auc_roc, sensitivity, specificity, balanced_accuracy, kappa, accuracy

In [ ]:
# Evaluate the model on test data
auc_roc_test, sensitivity_test, specificity_test, balanced_accuracy_test, kappa_test, accuracy = evaluate_model(rf_classifier, X_test, y_test)
print("\nTest Performance:")
print("AUC-ROC:", auc_roc_test)
print("Sensitivity:", sensitivity_test)
print("Specificity:", specificity_test)
print("Balanced Accuracy:", balanced_accuracy_test)
print("Cohen's Kappa:", kappa_test)
print("Accuracy:", accuracy)

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()

# Perform cross-validation
cv_scores = cross_val_score(logisticRegr, X_combined, y_combined, cv=5, scoring='accuracy')

# Print cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())

In [ ]:
logisticRegr.fit(X_combined, y_combined)

In [ ]:
# Evaluate the model on test data
auc_roc_test, sensitivity_test, specificity_test, balanced_accuracy_test, kappa_test, accuracy = evaluate_model(logisticRegr, X_test, y_test)
print("\nTest Performance:")
print("AUC-ROC:", auc_roc_test)
print("Sensitivity:", sensitivity_test)
print("Specificity:", specificity_test)
print("Balanced Accuracy:", balanced_accuracy_test)
print("Cohen's Kappa:", kappa_test)
print("Accuracy:", accuracy)

Support Vector Machine (SVM)

In [ ]:
from sklearn import svm
clf = svm.SVC()
# Perform cross-validation
cv_scores = cross_val_score(clf, X_combined, y_combined, cv=5, scoring='accuracy')

# Print cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())

In [ ]:
# Train the model
clf.fit(X_combined, y_combined)

In [ ]:
# Evaluate the model on test data
auc_roc_test, sensitivity_test, specificity_test, balanced_accuracy_test, kappa_test, accuracy = evaluate_model(clf, X_test, y_test)
print("\nTest Performance:")
print("AUC-ROC:", auc_roc_test)
print("Sensitivity:", sensitivity_test)
print("Specificity:", specificity_test)
print("Balanced Accuracy:", balanced_accuracy_test)
print("Cohen's Kappa:", kappa_test)
print("Accuracy:", accuracy)

Multilayer Perceptron (MLP) Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(210,800,600,500,100), activation='relu', solver='adam', max_iter=2000)

In [ ]:
mlp.fit(X_combined, y_combined)

In [ ]:
# Evaluate the model on test data
auc_roc_test, sensitivity_test, specificity_test, balanced_accuracy_test, kappa_test, accuracy = evaluate_model(mlp, X_test, y_test)
print("\nTest Performance:")
print("AUC-ROC:", auc_roc_test)
print("Sensitivity:", sensitivity_test)
print("Specificity:", specificity_test)
print("Balanced Accuracy:", balanced_accuracy_test)
print("Cohen's Kappa:", kappa_test)
print("Accuracy:", accuracy)

In [ ]:
predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))

Graph Neural Network (GNN)

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.8.0+cpu.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.8.0+cpu.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-1.8.0+cpu.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-1.8.0+cpu.html
!pip install torch-geometric

In [ ]:
def feature_gen_permeability(smiles):
  mol = Chem.MolFromSmiles(smiles)
  descriptors = []
  for descriptor_name, function in Descriptors.descList:
      descriptors.append(function(mol))
  descriptors = np.array(descriptors)
  return descriptors

In [ ]:
from torch_geometric.data import Data
import pandas as pd
import numpy as np
import torch
from torch_geometric.nn import GCNConv, global_mean_pool
from rdkit import Chem
import torch
import torch.nn as nn
from torch.optim import Adam

In [ ]:
import torch
from torch_geometric.nn import GCNConv, global_mean_pool

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(1, 30)
        self.conv2 = GCNConv(30, 50)
        self.fc1 = torch.nn.Linear(50 + 210, 30)
        self.fc2 = torch.nn.Linear(30, 2)

    def forward(self, data):
        x = self.conv1(data.x, data.edge_index)
        x = torch.relu(x)
        x = self.conv2(x, data.edge_index)
        x = torch.relu(x)
        x = global_mean_pool(x, data.batch)
  
        x = torch.cat([x, data.additional_features.unsqueeze(0)], dim=1)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
train_labels = torch.tensor(split['train']['Y'], dtype=torch.long)
train_features = []
for smiles in split['train']['Drug']:
    train_features.append(feature_gen_permeability(smiles))

train_data = []
for smiles, features in zip(split['train']['Drug'], train_features):
    train_data.append(molecule_to_graph(smiles, features))

In [ ]:
model = GNN()
optimizer = Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(1000):
    model.train()
    total_loss = 0
    for data, label in zip(train_data, train_labels):
        optimizer.zero_grad()
        output = model(data)
        label = torch.tensor([label], dtype=torch.long)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    average_loss = total_loss / len(train_data)
    print(epoch, average_loss)

0 19199.717566552783
1 0.5914562727596736
2 0.3915011842599076
3 0.3912443623857133
4 0.3912185822676323
5 0.39121232916381254
6 0.3912108046096865
7 0.39121041776549614
8 0.3912103354690199
9 0.39121031479156587
10 0.3912103093919878
11 0.3912103095908095
12 0.39121030711600285
13 0.39121030508069676
14 0.3912103050963932
15 0.3912103069433419
16 0.3912103069433419
17 0.3912103069433419
18 0.3912103069433419
19 0.3912103069433419
20 0.3912103069433419
21 0.3912103069433419
22 0.3912103069433419
23 0.3912103069433419
24 0.3912103069433419
25 0.3912103069433419
26 0.3912103069433419
27 0.3912103069433419
28 0.3912103069433419
29 0.3912103069433419
30 0.3912103069433419
31 0.3912103069433419
32 0.3912103069433419
33 0.3912103069433419
34 0.3912103069433419
35 0.3912103069433419
36 0.3912103069433419
37 0.3912103069433419
38 0.3912103069433419
39 0.3912103069433419
40 0.3912103069433419
41 0.3912103069433419
42 0.3912103069433419
43 0.3912103069433419
44 0.3912103069433419
45 0.3912103069

KeyboardInterrupt: 

In [ ]:
test_labels = torch.tensor(split['test']['Y'], dtype=torch.long)
test_features = []
for smiles in split['test']['Drug']:
    test_features.append(feature_gen_permeability(smiles))

test_data = []
for smiles, features in zip(split['test']['Drug'], test_features):
    test_data.append(molecule_to_graph(smiles, features))

In [ ]:
model.eval()
predictions = []

with torch.no_grad():
    for data in test_data:
        output = model(data)
        predictions.append(output.argmax(dim=1))

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Calculate metrics
precision = precision_score(test_labels, predictions)
recall = recall_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions)
accuracy = accuracy_score(test_labels, predictions)

print("accuracy:", accuracy)
print("precision:", precision)
print("recall:", recall)
print("f1:", f1)

accuracy: 0.8452088452088452
precision: 0.8452088452088452
recall: 1.0
f1: 0.9161118508655127


In [ ]:
def evaluate_model_gnn(model, X, y_pred):
    y = split['test']['Y']
    auc_roc = roc_auc_score(y, y_pred)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    balanced_accuracy = (sensitivity + specificity) / 2
    kappa = cohen_kappa_score(y, y_pred)
    accuracy = accuracy_score(y, y_pred)

    return auc_roc, sensitivity, specificity, balanced_accuracy, kappa, accuracy

In [ ]:
# Evaluate the model on test data
from sklearn.metrics import roc_auc_score, confusion_matrix, cohen_kappa_score, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

auc_roc_test, sensitivity_test, specificity_test, balanced_accuracy_test, kappa_test, accuracy = evaluate_model_gnn(model, test_labels, predictions)
print("\nTest Performance:")
print("AUC-ROC:", auc_roc_test)
print("Sensitivity:", sensitivity_test)
print("Specificity:", specificity_test)
print("Balanced Accuracy:", balanced_accuracy_test)
print("Cohen's Kappa:", kappa_test)
print("Accuracy:", accuracy)


Test Performance:
AUC-ROC: 0.5
Sensitivity: 1.0
Specificity: 0.0
Balanced Accuracy: 0.5
Cohen's Kappa: 0.0
Accuracy: 0.8452088452088452
